In [2]:
import pandas as pd

from lxutils import config
from fast_bitrix24 import BitrixAsync
from itertools import chain

from constants import Stage, Field

b = BitrixAsync(config['tokens']['webhook'])

deals = pd.DataFrame(await b.get_all('crm.deal.list', params={
    'select': ['ID', Field.REASON, 'STAGE_ID'],
    'filter': {'STAGE_ID': [Stage.LOST, Stage.POSTPONED]}  # TODO: добавить отложенные
    }))

timeline = await b.call(
    'crm.timeline.comment.list',
    [{
        'FILTER': {
            'ENTITY_TYPE': 'deal',
            'ENTITY_ID': deal,
        },
    } for deal in deals['ID']])

100%|██████████| 2019/2019 [00:01<00:00, 1192.61it/s]


In [2]:
deals['num_comments'] = [len(deal_timeline) for deal_timeline in timeline]
deals['latest_comment'] = [
    max(deal_timeline, key=lambda x: x['CREATED'])['CREATED']
    if deal_timeline else None
    for deal_timeline in timeline]

# отобрать тех, у кого статус - проиграна, было более-менее плотное общение,
# и прошло уже 2 мес.

deals = deals.query('num_comments >= 3 and latest_comment <= "2020-12-01"')

contact_refs = await b.get_by_ID('crm.deal.contact.items.get', deals['ID'])

contact_refs_chained = chain(*contact_refs.values())
IDs = [contact_ref['CONTACT_ID'] for contact_ref in contact_refs_chained]

contacts = await b.get_by_ID('crm.contact.get', IDs)

contacts_stripped = [{'name': cont['NAME'], 'phone': cont['PHONE']}
                     for cont in contacts.values() if 'PHONE' in cont]

 99%|█████████▉| 952/957 [00:03<00:00, 291.92it/s]


In [3]:
phones_unpacked = chain(*[
    [{'name': cont['name'], 'phone': phone['VALUE']}
     for phone in cont['phone']]
    for cont in contacts_stripped])

phones = pd.DataFrame(phones_unpacked)

In [6]:
# удаляем все символы, кроме цифр

phones['phone'] = [
    ''.join(char for char in phone if char in '0123456789')
    for phone in phones['phone']
]

# удаляем дефектных
phones['phone_len'] = [len(ph) for ph in phones['phone']]
phones = phones.query('phone_len == 11').drop(columns=['phone_len'])

# начальный знак - семерка
phones['phone'] = ['7' + ph[1:] if ph[0] == '8' else ph
                   for ph in phones['phone']]

# удаляем дубли

phones = phones.drop_duplicates('phone')
phones.shape[0]

869

In [5]:
phones.to_csv('phones.csv', index=False)